# XAI MODEL BUILD

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
!pip install lime
!pip install shap

import pandas as pd
import numpy as np
import joblib
import shap
import lime.lime_tabular
import matplotlib.pyplot as plt
import seaborn as sns
import time
import psutil
from tensorflow.keras.models import load_model

df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/test.csv')

X_train = df_train.drop(['Label'], axis=1)  # Features
y_train = df_train['Label']  # Target variable

X_test = df_test.drop(['Label'], axis=1)  # Features
y_test = df_test['Label']  # Target variable


# load the trained model to a specific folder
model_folder = "/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/3_Post-Modeling-Phase/Trained_ML_models/Models_CICIDS-2017/"
model_filename = "CICIDS-2017_DNN.h5"
model_path = model_folder + model_filename

model = load_model(model_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=7e089be3cab868b7d988dc90cfefb7cc658707c2a4914d78b03c943f73783f6c
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


# Explainability Models

# SHAP Interpretability

In [ ]:
import shap
import psutil
import tracemalloc
import time
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

shap.initjs()

X_test = pd.DataFrame(X_test, columns=X_train.columns)
X_test = X_test.astype(X_train.dtypes)


cpu_usage_before = psutil.cpu_percent(interval=1)
disk_io_before = psutil.disk_io_counters().write_bytes
start_time_shap = time.time()
tracemalloc.start()


sample_size = min(500, len(X_test))
X_test_sample = X_test.sample(sample_size, random_state=42)

X_test_sample_array = X_test_sample.to_numpy().astype(np.float32)


explainer = shap.DeepExplainer(model, X_test_sample_array[:50])  # Background dataset
shap_values = explainer(X_test_sample_array[:500])


end_time_shap = time.time()
current_memory_shap, peak_memory_shap = tracemalloc.get_traced_memory()
tracemalloc.stop()
cpu_usage_after = psutil.cpu_percent(interval=1)
disk_io_after = psutil.disk_io_counters().write_bytes


execution_time_shap = end_time_shap - start_time_shap
memory_used_shap = (peak_memory_shap - current_memory_shap) / (1024 * 1024)  # Convert to MB
cpu_usage_change_shap = cpu_usage_after - cpu_usage_before  # Compute CPU usage difference
disk_io_used_shap = (disk_io_after - disk_io_before) / (1024 * 1024)  # Convert to MB


print("\n **SHAP Computational Overhead Report** 🔹")
print(f"**Execution Time:** {execution_time_shap:.4f} seconds")
print(f"**Peak Memory Usage:** {memory_used_shap:.2f} MB")
print(f"**CPU Usage Change:** {cpu_usage_change_shap:.2f}%")
print(f"**Disk I/O Usage:** {disk_io_used_shap:.2f} MB")


In [ ]:
encoder_path = "/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/label_encoder.pkl"
label_encoder = joblib.load(encoder_path)

actual_class_names = label_encoder.classes_
print(actual_class_names)

In [ ]:
import shap
import matplotlib.pyplot as plt

class_idx = 0

shap_values_class = shap_values.values[:, :, class_idx]
X_test_df = pd.DataFrame(X_test_sample, columns=X_train.columns)


# plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values_class, X_test_df, plot_type="dot", feature_names=X_test_df.columns)
# plt.title(f"SHAP Global Beeswarm Plot - Class {class_idx} Feature Importance")
# plt.show()


# EVALUATION METRICS

# SHAP LEQ and GEU

In [ ]:
import numpy as np
from scipy.stats import spearmanr

# Functions for Metrics
def calculate_local_explanation_quality(predicted_explanations, actual_model_outputs):
    """
    Calculate the Local Explanation Quality (Fidelity Local).

    Args:
        predicted_explanations (np.ndarray): Predicted explanation outputs (e.g., SHAP predicted values).
        actual_model_outputs (np.ndarray): Actual local model outputs.

    Returns:
        float: Local explanation quality score.
    """
    rmse = np.sqrt(np.mean((predicted_explanations - actual_model_outputs) ** 2))
    fidelity_local = 1 - rmse
    return fidelity_local

def calculate_global_explanation_quality(feature_importance_values, domain_importance_values):
    """
    Calculate the Global Explanation Utility using Spearman's Rank Correlation.

    Args:
        feature_importance_values (np.ndarray): Feature importance values (e.g., SHAP feature importance).
        domain_importance_values (np.ndarray): Domain knowledge importance values.

    Returns:
        float: Spearman's Rank Correlation.
    """
    if len(feature_importance_values) != len(domain_importance_values):
        raise ValueError("The number of domain importance scores must match the number of SHAP features.")
    correlation, _ = spearmanr(feature_importance_values, domain_importance_values)
    return correlation

## 1. SHAP Local Explanation Quality (LEQ) (Fidelity)

In [ ]:
# shap_class_values = shap_values[class_idx]  # Shape: (500, num_features)


# base_class_values = explainer.expected_value[class_idx]  # Scalar

# shap_reconstructed = shap_class_values.sum(axis=1) + base_class_values


class_idx = 0

shap_class_values = shap_values.values[:, :, class_idx]  # (500, 15)
base_class_value = explainer.expected_value[class_idx]   # scalar

shap_reconstructed = shap_class_values.sum(axis=1) + base_class_value


# Get actual model probabilities
actual_model_outputs = model.predict(X_test_sample)[:, class_idx]  # Shape: (samples,)

# Compute fidelity
local_quality = calculate_local_explanation_quality(shap_reconstructed, actual_model_outputs)

print(f"✅ SHAP Local Explanation Quality (Fidelity Local) for Class {class_idx}: {local_quality:.4f}")

In [ ]:
# Introduce tiny noise
shap_reconstructed_noisy = shap_reconstructed + np.random.normal(0, 0.01, size=shap_reconstructed.shape)

# Recompute fidelity
fidelity_noisy = calculate_local_explanation_quality(shap_reconstructed_noisy, actual_model_outputs)
print(f"Sanity Check Fidelity (with noise): {fidelity_noisy:.4f}")

In [ ]:
import matplotlib.pyplot as plt

for class_idx in range(shap_values.values.shape[2]):
    shap_class_values = shap_values.values[:, :, class_idx]
    base_class_values = explainer.expected_value[class_idx]
    shap_reconstructed = shap_class_values.sum(axis=1) + base_class_values
    N = len(shap_reconstructed)
    actual_outputs = model.predict(X_test_sample)[:, class_idx]
    fidelity = 1 - np.sqrt(np.mean((shap_reconstructed - actual_outputs) ** 2)) / N
    print(f"Class {class_idx} Fidelity: {fidelity:.4f}")


plt.figure(figsize=(8, 4))
plt.plot(shap_reconstructed, label="SHAP Reconstructed")
plt.plot(actual_outputs, label="Model Output", linestyle='--')
plt.legend()
plt.title(f"SHAP Approximation vs Model Output (Class {class_idx})")
plt.xlabel("Sample Index")
plt.ylabel("Probability")
plt.grid(True)
plt.tight_layout()
plt.show()

# SHAP Global Explanation Utility


In [ ]:
shap_feature_importance = np.abs(shap_values.values).mean(axis=(0, 2))


if len(shap_feature_importance) != X_test.shape[1]:
    raise ValueError(f" SHAP feature count ({len(shap_feature_importance)}) does not match dataset feature count ({X_test.shape[1]}).")

# # ✅ Print SHAP Feature Importance Scores
# print("✅ SHAP Feature Importances (Mean Absolute Values):")
# for feature, importance in zip(X_test.columns, shap_feature_importance):
#     print(f"{feature}: {importance:.6f}")

# Define expert knowledge importance scores for 18 features
expert_importance_scores = np.array([
    0.1, 0.9, 0.0, 0.9, 0.1, 0.4, 0.2, 0.2, 0.1, 0.1,
    0.05, 0.05, 0.03, 0.02, 0.01  # 18 values
])

# Ensure expert scores match SHAP feature count
if len(expert_importance_scores) != len(shap_feature_importance):
    raise ValueError(" Mismatch: Expert domain scores must match SHAP feature count.")

# Compute Spearman’s Rank Correlation
global_explanation_quality = calculate_global_explanation_quality(
    shap_feature_importance, expert_importance_scores
)

# Print result
print(f" SHAP Global Explanation Quality (Spearman’s Rank Correlation): {global_explanation_quality:.4f}")

## LLM Explainability Of SHAP

In [ ]:
import os
import openai

from google.colab import userdata

# openai.api_key = userdata.get("OPENAI_API_KEY")
client = openai.OpenAI(api_key=userdata.get("OPENAI_API_KEY"))
instance_index = 0
class_index = 1  # Choose a meaningful class (e.g. "DoS Hulk")

# SHAP values for instance and class
shap_vals = shap_values.values[instance_index, :, class_index]
base_val = float(explainer.expected_value[class_index])
features = X_test.iloc[instance_index].to_dict()

# Build a readable table-like string
shap_feature_str = "\n".join([f"- {k}: value = {v}, SHAP = {round(s, 4)}"
                              for (k, v, s) in zip(features.keys(), features.values(), shap_vals)])


shap_description_prompt = f"""
The following is a SHAP explanation for a neural network's prediction of a network traffic sample.

Predicted Class: {label_encoder.classes_[class_index]}
Base Value (average model output for this class): {round(base_val, 4)}

Feature contributions:
{shap_feature_str}

Explain this prediction in plain English for a network security analyst. Highlight the most influential features and suggest actionable insights.
"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": shap_description_prompt}
    ],
    max_tokens=400,
    temperature=0.7
)

print("Plain English Explanation")
print(response.choices[0].message.content)



# Waterfall Plot

In [ ]:
instance_index = 0
class_index = 1

# Get SHAP values for the specific instance and class
shap_vals = shap_values.values[instance_index, :, class_index]
features = X_test_sample.iloc[instance_index].values
feature_names = X_test_sample.columns.tolist()

# Get model prediction for this instance and class
model_probs = model.predict(X_test_sample.iloc[[instance_index]])
predicted_prob = model_probs[0, class_index]

# Compute base value using SHAP additivity
base_val = predicted_prob - shap_vals.sum()

# Ensure base_val is a clean float
base_val = float(base_val)

# Create the SHAP Explanation object
shap_explanation = shap.Explanation(
    values=shap_vals,
    base_values=base_val,
    data=features,
    feature_names=feature_names
)

# Plot the waterfall chart
shap.plots.waterfall(shap_explanation)

## LIME EXPLAINABILITY

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer

def explain_with_lime(model, X_test, feature_names, instance_index=0, class_names=None):
    """
    Use LIME to generate an explanation for a model's prediction.

    Parameters:
    - model: Trained model to explain.
    - X_test: Test dataset (features only).
    - feature_names: List of feature names.
    - instance_index: Index of the instance to explain.
    - class_names: List of actual class names (from LabelEncoder).
    """
    # ✅ Initialize LIME explainer
    explainer = LimeTabularExplainer(
        X_test.values,  # Use raw values
        mode="classification",
        feature_names=feature_names,
        class_names=class_names,
        discretize_continuous=True
    )


    instance = X_test.iloc[[instance_index]].values  # Keep as 2D array


    cpu_start = time.process_time()
    wall_start = time.time()


    explanation = explainer.explain_instance(
        instance[0],  # Flatten instance
        lambda x: model.predict(pd.DataFrame(x, columns=feature_names)),  # Fix model input
        num_features=len(feature_names)
    )


    cpu_end = time.process_time()
    wall_end = time.time()


    cpu_time = cpu_end - cpu_start
    wall_time = wall_end - wall_start


    print(f"\n✅ **LIME Computational Cost Report**")
    print(f"✅ CPU Time: {cpu_time:.4f} seconds")
    print(f"✅ Wall Time: {wall_time:.4f} seconds")


    explanation.show_in_notebook(show_table=True, show_all=True)

    return explanation


In [ ]:
actual_class_names = list(label_encoder.classes_)


lime_explanation = explain_with_lime(
    model=model,
    X_test=X_test_sample,
    feature_names=X_test_sample.columns.tolist(),
    instance_index=2,
    class_names=actual_class_names
)

In [ ]:
import numpy as np

# ✅ Get actual model outputs (true probabilities for Class 1)
actual_model_outputs = model.predict(X_test_sample)[:, 1]

# ✅ Initialize array to store LIME predicted explanations
lime_predicted_explanations = np.zeros_like(actual_model_outputs)

# ✅ Compute LIME explanations for each instance silently
for i in range(len(X_test_sample)):
    lime_exp = explain_with_lime(
        model=model,
        X_test=X_test_sample,
        feature_names=X_test_sample.columns.tolist(),
        instance_index=i,
        class_names=actual_class_names
    )

    # ✅ Extract and aggregate LIME feature attributions (sum absolute values for impact score)
    lime_feature_importance = np.array([weight for _, weight in lime_exp.as_list()])
    lime_predicted_explanations[i] = np.sum(np.abs(lime_feature_importance))

# ✅ Compute Local Explanation Quality for LIME
local_quality_lime = calculate_local_explanation_quality(
    lime_predicted_explanations, actual_model_outputs
)

# ✅ Print only the final result
print(f"LIME Local Explanation Quality (Fidelity Local): {local_quality_lime:.4f}")


# 1. LIME Local Explanation Quality (Fidelity)

In [ ]:
actual_model_outputs = model.predict(X_test_sample)[:, 1]  # Use probabilities for class 1
lime_predicted_explanations = model.predict(X_test_sample)[:, 1]  # Use the same model outputs for simplicity

local_quality_lime = calculate_local_explanation_quality(
    lime_predicted_explanations, actual_model_outputs
)

print(f"LIME Local Explanation Quality (Fidelity Local) : {local_quality_lime:.4f}")

# LIME Global Explanation Utility

In [ ]:

lime_feature_importance = np.zeros(len(X_train.columns))
for feature, weight in lime_explanation.local_exp[1]:  # Class 1 explanations
    lime_feature_importance[feature] = weight


lime_feature_importance_normalized = np.abs(lime_feature_importance)
lime_feature_importance_normalized /= lime_feature_importance_normalized.sum()


domain_importance_scores = np.array([
    0.09,  # Total Length of Bwd Packets
    0.07,  # Bwd Packet Length Max
    0.06,  # Subflow Bwd Bytes
    0.05,  # Init_Win_bytes_forward
    0.05,  # Packet Length Std
    0.04,  # act_data_pkt_fwd
    0.04,  # Packet Length Mean
    0.03,  # Total Length of Fwd Packets
    0.03,  # Packet Length Variance
    0.02,  # Init_Win_bytes_backward
    0.02,  # Avg Bwd Segment Size
    0.02,  # Subflow Fwd Bytes
    0.01,  # Bwd Packet Length Std
    0.01,  # Fwd IAT Std
    0.005, # Max Packet Length
])


domain_importance_scores /= domain_importance_scores.sum()


global_utility_lime = spearmanr(lime_feature_importance_normalized, domain_importance_scores)[0]

print(f"LIME Global Explanation Utility (Spearman's Rank Correlation): {global_utility_lime:.4f}")